In [10]:
import pandas as pd
df = pd.read_csv('/Users/vigneshramesh/Desktop/Deals/Natwest/question_tree.csv')

In [11]:
df.head(10)

,Question ID,Question Category,Question Text,Parent Question ID,Parent Answer Trigger,Answer Type,Options,Is Multi-select,Some GT Available,Questions to Legal Team
0,1,Agreement Details,Governing law,NaN,NaN,NaN,Not Specified|England and Wales|Scotland|North...,No,Yes,NaN
1,2,Agreement Details,Please detail the clause number of the governi...,NaN,NaN,Free Text,NaN,No,Yes,NaN
2,3,Eligibility Part 1,Does the agreement contain express provisions ...,NaN,NaN,Single Select,Yes|No,No,Yes,NaN
3,4,Eligibility Part 1,Is sub-participation defined?,3.0,Yes,Single Select,Yes|No,No,Yes,NaN
4,5,Eligibility Part 1,Please set out the relevant definition of sub-...,4.0,Yes,Free Text,NaN,No,Yes,NaN
5,6,Eligibility Part 1,Does the definition of sub-participation captu...,4.0,Yes,Single Select,No - definition only relates to transfer of vo...,No,Yes,How is this evaluated? Are we looking for spec...
6,7,Eligibility Part 1,Select what conditions apply to the Lender's r...,6.0,Yes,Multi Select,None|Borrower/Parent/Obligors consent|Borrower...,Yes,Yes,NaN
7,8,Eligibility Part 1,Please detail what restrictions apply,7.0,"Conditions relating to Permitted Lenders, Rest...",Free Text,NaN,No,Yes,NaN
8,9,Eligibility Part 1,What other conditions apply?,7.0,Other,Free Text,NaN,No,NaN,NaN
9,10,Eligibility Part 1,Select all conditions that apply to Borrower's...,7.0,Borrower/Parent/Obligors consent,Multi Select,Not unreasonably withheld|Not unreasonably del...,Yes,NaN,NaN


In [22]:
# Reference: https://ai.google.dev/gemini-api/docs/caching?lang=python

import os
import google.generativeai as genai
from google.generativeai import caching
import datetime
import time

from dotenv import load_dotenv
load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)

# model = genai.GenerativeModel("gemini-1.5-flash")
# response = model.generate_content("How does AI work?")
# print(response.text)

pdf_file = genai.upload_file('/Users/vigneshramesh/Desktop/Deals/Natwest/Baseline Files/term-loan-facility-agreement.pdf')

In [104]:
SYSTEM_PROMPT = """
You are to review a provided facility agreement in context to determine how the agreement affects the inclusion of assets in Significant Risk Transfers (SRTs).
Your purpose is to identify provisions regarding confidentiality, transferability, consents, confidentiality undertakings, approved lender lists, and any conditions that must be satisfied for including the asset in an SRT.
Your final output must be sufficiently clear, detailed, and legally accurate so that a legal team can rely on it directly.

In cases where the options for you to respond with are explicitly provided in the user prompt, you are required to respond with the most appropriate option and nothing else in your response.
"""

In [77]:
import PyPDF2
import base64

with open('/Users/vigneshramesh/Desktop/Deals/Natwest/Baseline Files/revolving_facility_agreement_18-07-23.pdf', 'rb') as f:
    pdf = PyPDF2.PdfReader(f)
    # pdf_data = base64.standard_b64encode(f.read()).decode("utf-8")
    num_pages = len(pdf.pages)
    text = ''
    for page in range(num_pages):
        text += pdf.pages[page].extract_text()

In [105]:
import anthropic
import os

from dotenv import load_dotenv
load_dotenv()
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')

client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

In [106]:
messages = []
system = [
    {
        "type": "text",
        "text": SYSTEM_PROMPT,
        "cache_control": {"type": "ephemeral"}
    }
    ]

messages.append(
    {
        "role": "user",
        "content": [
            {
        "type": "document",
        "source": {
            "type": "text",
            "media_type": "text/plain",
            "data": text,
        },
        "title": "Revolving Facility Agreement 18-07-23",
        "context": "This is a Revolving Facility Agreement.",
        "citations": {"enabled": True},
        "cache_control": {"type": "ephemeral"}
    },
            {
            "type": "text",
            "text": "Please review the provided document and provide your response.",
            }
        ]
    }
)

for idx, row in df.iterrows():
    question = row['Question ']
    section_title = row['Title']
    answer_options = row['Answer Options '] if row['Answer Options '] != 'BLANK' else None
    if answer_options:
        user_prompt = f"\n\nQuestion Category: {section_title}\nUser Question/Request for Details: {question}?\nYour Answer Options are: \n{answer_options}"
    else:
        user_prompt = f"\n\nQuestion Category: {section_title}\nUser Question/Request for Details: {question}"
    messages.append({"role": "user", "content": [{"type":"text", "text":user_prompt, "cache_control": {"type": "ephemeral"}}]})
    response = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=1024,
        system=system,
        messages=messages
    )
    df.at[idx, 'Answer'] = response.content[0].text
    df.at[idx, 'Citation'] = response.content[0].citations
    messages[-1]['content'][0].pop('cache_control')
    messages.append({"role": "assistant", "content": response.content[0].text})
    

In [115]:
for idx, row in df.iterrows():
    q_num = idx + 1
    df.at[idx, 'Question Number'] = q_num
    if row['Citation'] == None:
        df.at[idx, 'Citation'] = 'No Citations'

In [116]:
df.to_csv('/Users/vigneshramesh/Desktop/Deals/Natwest/BaselineClaude3.5Responses.csv', index=False)